### **Callback API**
- 모델이 학습 중에 충돌이 발생하거나 네트워크가 끊기면, 모든 훈련 시간이 낭비될 수 있고,
  과적합을 방지하기 위해 훈련을 중간에 중지해야 할 수도 있다.
- 모델이 학습을 시작하면 학습이 완료될 때까지 아무런 제어를 하지 못하게 되고,
  신경망 훈련을 완료하는 데에는 몇 시간 또는 며칠이 걸릴 수 있기 때문에 모델을 모니터링하고 제어할 수 있는 기능이 필요하다.
- 훈련시(fit()) Callback API를 등록시키면 반복 내에서 특정 이벤트 발생마다 등록된 callback이 호출되어 수행된다.

**ModelCheckpoint(filepath, monitor='val_loss', valbose=0, save_best_only=False, save_weight_only=False, mode='auto')**
- 특정 조건에 따라서 모델 또는 가중치를 파일로 저장한다.
- filepath: " weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.hdf5 "와 같이 모델의 체크포인트를 저장한다.
- monitor: 모니터링할 성능 지표를 작성한다
- save_best_only: 가장 좋은 성능을 나타내는 모델을 저장할지에 대한 여부
- save_weight_only: weights만 저장할지에 대한 여부
- mode: {auto, min, max} 중 한 가지를 작성한다. monitor의 성능 지표에 따라 좋은 경우를 선택한다.  
  <sub>monitor의 성능 지표가 감소해야 좋은 경우 min, 증가해야 좋은 경우 max, monitor의 이름으로부터 자동으로 유추하고 싶다면 auto</sub>

**ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, vervose=0, mode='auto', min_lr=0)**
- 특정 반복동안 성능이 개선되지 않을 때, 학습률을 동적으로 감소시킨다.
- monitor: 모니터링할 성능 지표를 작성한다.
- factor: 학습률을 감소시킬 비율, 새로운 학습률 = 기존 학습률 * factor
- patience: 학습률을 줄이기 전에 monitor할 반복 횟수
- mode: {auto, min, max} 중 한 가지를 작성한다. monitor의 성능 지표에 따라 좋은 경우를 선택한다.  
  <sub>monitor의 성능 지표가 감소해야 좋은 경우 min, 증가해야 좋은 경우 max, monitor의 이름으로부터 자동으로 유추하고 싶다면 auto</sub>

**EarlyStopping(monitor='val_loss', patience=0, vervose=0, mode='auto')**
- 특정 반복동안 성능이 개선되지 않을 때, 학습을 조기에 중단한다.
- monitor: 모니터링할 성능 지표를 작성한다.
- patience: Early Stopping을 적용하기 전에 monitor할 반복 횟수.
- mode: {auto, min, max} 중 한 가지를 작성한다. monitor의 성능 지표에 따라 좋은 경우를 선택한다.  
  <sub>monitor의 성능 지표가 감소해야 좋은 경우 min, 증가해야 좋은 경우 max, monitor의 이름으로부터 자동으로 유추하고 싶다면 auto</sub>

In [1]:
# 모니터에 뭘 넣는지에 따라 모드가 날라진다

In [2]:
from tensorflow.keras.layers import Layer, Input, Dense, Flatten
from tensorflow.keras.models import Model

INPUT_SIZE = 28

def create_model():
    input_tensor = Input(shape=(INPUT_SIZE, INPUT_SIZE))
    x = Flatten()(input_tensor)
    x = Dense(64, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(10, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=output)
    return model

In [3]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

def get_preprocessed_data(images, targets):
    images = np.array(images / 255.0, dtype=np.float32)
    targets = np.array(targets, dtype=np.float32)

    return images, targets

def get_preprocessed_ohe(images, targets):
    images, targets = get_preprocessed_data(images, targets)
    oh_targets = to_categorical(targets)

    return images, oh_targets

def get_train_valid_test(train_images, train_targets, test_images, test_targets, validation_size=0.2, random_state=124):
    train_images, train_oh_targets = get_preprocessed_ohe(train_images, train_targets)
    test_images, test_oh_targets = get_preprocessed_ohe(test_images, test_targets)

    train_images, validation_images, train_oh_targets, validation_oh_targets = \
    train_test_split(train_images, train_oh_targets, stratify=train_oh_targets, test_size=validation_size, random_state=random_state)

    return (train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets)

In [4]:
from tensorflow.keras.datasets import fashion_mnist

(train_images, train_targets), (test_images, test_targets) = fashion_mnist.load_data()

(train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets) = \
get_train_valid_test(train_images, train_targets, test_images, test_targets)

print(train_images.shape, train_oh_targets.shape)
print(validation_images.shape, validation_oh_targets.shape)
print(test_images.shape, test_oh_targets.shape)

(48000, 28, 28) (48000, 10)
(12000, 28, 28) (12000, 10)
(10000, 28, 28) (10000, 10)


In [5]:
!dir

 E 드라이브의 볼륨: 새 볼륨
 볼륨 일련 번호: 6A3C-E039

 E:\kdt_0900_pjw\ai\deep_learning\c_tensorflow 디렉터리

2024-05-28 화  오후 05:13    <DIR>          .
2024-05-28 화  오전 11:29    <DIR>          ..
2024-05-28 화  오후 04:53    <DIR>          .ipynb_checkpoints
2024-05-28 화  오후 05:13            92,902 a_tensorflow.ipynb
2024-05-28 화  오후 05:13            33,450 b_keras.ipynb
2024-05-28 화  오전 11:22           162,362 b_keras_task.ipynb
2024-05-28 화  오전 11:09    <DIR>          callback_files
2024-05-28 화  오전 11:22            17,378 c_callback.ipynb
2024-05-28 화  오후 04:26            14,985 c_callback_task.ipynb
2024-05-27 월  오후 03:26    <DIR>          images
2024-05-28 화  오후 04:53                72 Untitled.ipynb
               6개 파일             321,149 바이트
               5개 디렉터리  488,045,563,904 바이트 남음


In [6]:
!dir callback_files

 E 드라이브의 볼륨: 새 볼륨
 볼륨 일련 번호: 6A3C-E039

 E:\kdt_0900_pjw\ai\deep_learning\c_tensorflow\callback_files 디렉터리

2024-05-28 화  오전 11:09    <DIR>          .
2024-05-28 화  오후 05:13    <DIR>          ..
2024-05-28 화  오전 11:09           739,600 weights.001-0.1798-0.9015.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.002-0.1453-0.9571.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.003-0.1256-0.9682.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.004-0.1084-0.9741.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.005-0.0909-0.9791.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.006-0.1047-0.9825.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.007-0.0865-0.9856.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.008-0.0943-0.9868.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.009-0.1112-0.9900.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.010-0.0804-0.9954.weights.h5
2024-05-28 화  오전 11:09   

### ModelCheckpoint

In [7]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

# weight 저장하기
# 생성자를 연다. ModelCheckpoint
# 꼭 담아줘야한다.
# 담은 후에 fit에 넣어준다.
mcp_cb = ModelCheckpoint(
    filepath="./callback_files/weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.h5",
    monitor='val_loss',
    # 모든 epoch의 파일을 저장하지 않고 좋은 성능이라 판단될 경우만 저장할 때 True설정
    save_best_only=False,
    save_weights_only=True,
    mode='min'
)

# model 저장하기
# mcp_cb = ModelCheckpoint(
#     filepath="./callback_files/model.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.model.keras",
#     monitor='val_loss',
#     # 모든 epoch의 파일을 저장하지 않고 좋은 성능이라 판단될 경우만 저장할 때 True설정
#     save_best_only=False,
#     save_weights_only=False,
#     mode='min'
# )

# callback 파라미터 전달
history = model.fit(x=train_images, y=train_oh_targets, validation_data=(validation_images, validation_oh_targets), batch_size=64, epochs=20, callbacks=[mcp_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - acc: 0.7298 - loss: 0.7630 - val_acc: 0.8453 - val_loss: 0.4312
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8571 - loss: 0.3998 - val_acc: 0.8612 - val_loss: 0.3702
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8734 - loss: 0.3514 - val_acc: 0.8682 - val_loss: 0.3616
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8833 - loss: 0.3207 - val_acc: 0.8705 - val_loss: 0.3544
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8896 - loss: 0.3042 - val_acc: 0.8692 - val_loss: 0.3467
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8873 - loss: 0.2958 - val_acc: 0.8798 - val_loss: 0.3288
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8976 - loss: 0.2796 - val_acc: 0.8716 - val_loss: 0.3434
Epoch 8/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.9002 - loss: 0.2675 - val_acc: 0.8842 - val_loss: 0.3169
Epoch 9/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - ac

In [8]:
# save_best_only 를 False로 주면 전부 다 저장 True를 주면 가장 좋은 성능이라 판단될 경우만 저장
# save_weight_only 를 False 로 주면 모델을 저장

# weight를 파일로 내보내는 이유: 모델을 내보내면 그걸 그대로 프리딕트 하겠다는 뜻. 웨이트를 내보내면 거기서부터 시작.
# 그걸 가져다가 본인들의 이미지로 훈련시키겠다는 뜻. 사용하는 용도가 더 많아짐.
# 모델의 층까지 동일하게 쓰고싶다 = keras 내보내기

In [9]:
# 저장된 파일 확인
!dir callback_files

 E 드라이브의 볼륨: 새 볼륨
 볼륨 일련 번호: 6A3C-E039

 E:\kdt_0900_pjw\ai\deep_learning\c_tensorflow\callback_files 디렉터리

2024-05-28 화  오후 05:26    <DIR>          .
2024-05-28 화  오후 05:25    <DIR>          ..
2024-05-28 화  오전 11:09           739,600 weights.001-0.1798-0.9015.weights.h5
2024-05-28 화  오후 05:25           739,600 weights.001-0.4312-0.8026.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.002-0.1453-0.9571.weights.h5
2024-05-28 화  오후 05:25           739,600 weights.002-0.3702-0.8593.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.003-0.1256-0.9682.weights.h5
2024-05-28 화  오후 05:25           739,600 weights.003-0.3616-0.8725.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.004-0.1084-0.9741.weights.h5
2024-05-28 화  오후 05:25           739,600 weights.004-0.3544-0.8816.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.005-0.0909-0.9791.weights.h5
2024-05-28 화  오후 05:25           739,600 weights.005-0.3467-0.8881.weights.h5
2024-05-28 화  오전 11:09   

In [10]:
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 799us/step - acc: 0.8817 - loss: 0.3720


[0.36918577551841736, 0.8828999996185303]

In [11]:
# create_model 초기화
model = create_model()

# weights 불러오기
# model.load_weights(경로)
model.load_weights('./callback_files/weights.020-0.3282-0.9295.weights.h5')

# compile
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

# test 확인
model.evaluate(test_images, test_oh_targets)

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = './callback_files/weights.020-0.3282-0.9295.weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
model.evaluate(test_images, test_oh_targets)

In [ ]:
from tensorflow.keras.models import load_model
# model 불러오기
# model = load_model(경로)
model = load_model('./callback_files/model.020-0.3350-0.9287.model.keras')

# test 확인
model.evaluate(test_images, test_oh_targets)

### ReduceLROnPlateau

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

# 생성자를 연다. ReduceLROnPlateau
# 꼭 담아줘야한다. 담은 후에 fit에 넣어준다.
rlr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    mode='min'
)

# callback 파라미터에 전달
history = model.fit(x=train_images, y=train_oh_targets, validation_data=(validation_images, validation_oh_targets), batch_size=64, epochs=20, callbacks=[rlr_cb])

### EarlyStopping

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc']

# 생성자를 연다. EarlyStopping
# 꼭 담아줘야한다. 담은 후에 fit에 넣어준다.
ely_cb = EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min'
)

# callback 파라미터에 전달
history = model.fit(x=train_images, y=train_oh_targets, validation_data=(validation_images, validation_oh_targets), batch_size=64, epochs=20, callbacks=[ely_cb])

### 세개 다 적용하기

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

mcp_cb = ModelCheckpoint(
    filepath="./callback_files/weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.h5",
    monitor='val_loss',
    # 모든 epoch의 파일을 저장하지 않고 좋은 성능이라 판단될 경우만 저장할 때 True설정
    save_best_only=False,
    save_weights_only=True,
    mode='min'
)

rlr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    mode='min'
)

ely_cb = EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min'
)

history = model.fit(x=train_images, y=train_oh_targets, validation_data=(validation_images, validation_oh_targets), batch_size=64, epochs=20, callbacks=[mcp_cb, rlr_cb, ely_cb])